## Conceito

A Otimização Preditiva é um recurso baseado em IA que lida automaticamente com as operações de manutenção das tabelas gerenciadas pelo Unity Catalog, eliminando a necessidade de ajustes manuais, melhorando o desempenho das consultas e reduzindo os custos de armazenamento.


**- OPTIMIZE:** para acionar o agrupamento incremental em tabelas habilitadas. Se o Liquid clustering automático estiver habilitado, a otimização preditiva poderá selecionar novas chaves de agrupamento antes de agrupar os dados.

**- VACUUM:** para reduzir os custos de armazenamento, excluindo arquivos de dados que não são mais referenciados pela tabela.

**- ANALYZE:** para acionar a atualização incremental das estatísticas da tabela. Essas estatísticas são usadas pelo otimizador de consultas para gerar um plano de consulta ideal. Consulte ANALYZE TABLE.

## Ativando a Otimização Preditiva

In [0]:
%sql
-- Ativar em uma tabela específica
ALTER TABLE seu_catalogo.seu_schema.sua_tabela 
SET PREDICTIVE OPTIMIZATION { ALWAYS | INHERIT };

-- Ativar em todo um esquema (recomendado para seus projetos de ETL)
ALTER SCHEMA seu_catalogo.seu_schema 
ENABLE PREDICTIVE OPTIMIZATION;

-- Ativar em todo o catálogo
ALTER CATALOG seu_catalogo 
ENABLE PREDICTIVE OPTIMIZATION;

## OPTIMIZE (Com Liquid Clustering)

Se a sua tabela foi criada com CLUSTER BY, o comando OPTIMIZE não precisa de parâmetros adicionais. A IA do Databricks decidirá quais arquivos compactar para melhorar a leitura.

In [0]:
%sql
-- Execução manual para forçar a organização imediata
OPTIMIZE seu_catalogo.seu_schema.sua_tabela;

## VACUUM (Limpeza de Arquivos)
Este comando remove arquivos físicos que não são mais necessários para versões antigas da tabela (Time Travel). Por padrão, o Delta mantém 7 dias de histórico.

In [0]:
%sql
-- Remove arquivos que não pertencem à versão atual há mais de 168 horas (7 dias)
VACUUM seu_catalogo.seu_schema.sua_tabela;

-- Se precisar limpar com menos de 7 dias (cuidado: rompe o Time Travel curto)
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
VACUUM seu_catalogo.seu_schema.sua_tabela RETAIN 24 HOURS;

## ANALYZE (Coleta de Estatísticas)
Este comando é vital para que o otimizador de consultas (CBO) saiba como distribuir as tarefas de processamento. Ele coleta informações sobre a distribuição dos dados em cada coluna.

In [0]:
%sql
-- Coleta estatísticas para todas as colunas da tabela
ANALYZE TABLE seu_catalogo.seu_schema.sua_tabela 
COMPUTE STATISTICS FOR ALL COLUMNS;